In [4]:
## import of the pandas library
## questions for myself: could this be done recursively? Get the books on one level and then go to other levels? Transitive...
import pandas as pd
import pickle

In [6]:
with open('C:/Users/ASUS/anaconda3/data_scentics/data/interim/books_data.pkl', 'rb') as f:
    books_data = pickle.load(f)

with open('C:/Users/ASUS/anaconda3/data_scentics/data/interim/ratings_data.pkl', 'rb') as f:
    ratings_data= pickle.load(f)

with open('C:/Users/ASUS/anaconda3/data_scentics/data/interim/users_data.pkl', 'rb') as f:
    users_data= pickle.load(f)

with open('C:/Users/ASUS/anaconda3/data_scentics/data/interim/books_ratings.pkl', 'rb') as f:
    books_ratings= pickle.load(f)


In [9]:
def recommendation (BookIsLiked, BookTitle, BookAuthor):

    ## find the Lord of the Rings books
    filtered_books = books_data.loc[(books_data["Book-Title"].str.contains(BookTitle)) & (books_data["Book-Author"].str.contains(BookAuthor))]
    filtered_books

    ## getting the users that like Lord of the Rings
    users_filtered = ratings_data.loc[(ratings_data['ISBN'].isin(filtered_books['ISBN'])) & (ratings_data['Book-Rating'] > BookIsLiked)]
    lord_readers = pd.unique(users_filtered['User-ID'])
    lord_readers

    ## getting the ISBN and Book-Ratings of the books that Lord of the Rings lovers also read
    books = ratings_data[ratings_data['User-ID'].isin(lord_readers)].reindex(columns = ['ISBN', 'Book-Rating'])
    ##books

    ## keeping only the books where there is more than 5 ratings
    count = books.groupby('ISBN')['Book-Rating'].count().reset_index(name = 'Count')
    count = count.loc[count["Count"] > 5]

    ## keeping only the books where the mean of the ratings is bigger than BookIsLiked, but the book should not be Lord of the Rings book
    ratings = pd.merge(left=ratings_data,right=count, left_on="ISBN", right_on="ISBN")
    ratings = ratings.groupby('ISBN')['Book-Rating'].mean().reset_index(name = 'MeanRating').sort_values(by='MeanRating', ascending=False)
    ratings = ratings.loc[ratings["MeanRating"] > BookIsLiked]
    ratings = ratings[ratings.ISBN.isin(filtered_books["ISBN"])==False]

    ##ratings
    return books_data[books_data['ISBN'].isin(ratings['ISBN'])]

In [20]:
## defining variables
BookIsLiked = 4 
BookTitle = 'THE LOVELY BONES A NOVEL'.upper()
BookAuthor = 'ALICE SEBOLD'.upper()

In [21]:
recommendation (BookIsLiked, BookTitle, BookAuthor)

,ISBN,Book-Author,Book-Title,Year-Of-Publication,Publisher,Language
37,0446310786,HARPER LEE,TO KILL A MOCKINGBIRD,1988,LITTLE BROWN &AMP; COMPANY,EN
38,0449005615,LAURA HILLENBRAND,SEABISCUIT AN AMERICAN LEGEND,2002,BALLANTINE BOOKS,EN
73,0140067477,BENJAMIN HOFF,THE TAO OF POOH,1983,PENGUIN BOOKS,EN
90,0316769487,J.D. SALINGER,THE CATCHER IN THE RYE,1991,"LITTLE, BROWN",EN
101,0446677450,ROBERT T. KIYOSAKI,"RICH DAD, POOR DAD WHAT THE RICH TEACH THEIR K...",2000,WARNER BOOKS,EN
...,...,...,...,...,...,...
65641,0316155306,MICHAEL CONNELLY,THE NARROWS A NOVEL,2004,"LITTLE, BROWN",EN
66354,0425192032,DEAN KOONTZ,LIGHTNING,2003,BERKLEY,EN
69152,1880418568,STEPHEN KING,"WOLVES OF THE CALLA THE DARK TOWER, BOOK 5",2003,DONALD M. GRANT/SCRIBNER,EN
71591,0060012781,ANTHONY BOURDAIN,A COOK'S TOUR GLOBAL ADVENTURES IN EXTREME CU...,2002,ECCO,EN
